# Level 1

## What is level 1

This part is a tutorial type introduction. We assume users have read [Aviary User Interface](../user_guide/user_interface.md). In this doc page we discuss Level 1's interface, but also other details, such as inputs and outputs.

If you have not yet, make sure to [install Aviary](installation.md).

To run Aviary in the Level 1 command-line interface (CLI), type this in a terminal:

In [ ]:
!aviary

We'll now discuss the different parts of the CLI.

## Aviary fortran_to_aviary command

```{note}
You only need to use the `fortran_to_aviary` command if you are converting a FLOPS or GASP deck to Aviary's csv format.
```

This [`fortran_to_aviary`](../user_guide/aviary_commands.ipynb) tool converts FORTRAN namelists into Aviary's csv based format using the mappings found in the `historical_name` section of the [variable_meta_data](../user_guide/variable_metadata.ipynb). The resulting csv is automatically sorted into three sections:
1. **Input Values:**
    Any FORTRAN variables that were mapped to input variables in Aviary components
2. **Initial Guesses:**
    Some variables are only used as initial guesses for the trajectory.
    These are displayed separately from the Input Values because they will not be passed directly to components
3. **Unconverted Values:**
    If the fortran_to_aviary converter can't find an Aviary variable that matches the FORTRAN variable, it is added to the end of the csv file.
    We recommend that you check this section after converting a namelist to ensure that there aren't any variables you expected to be converted here.
    Many of these unconverted variables represent features or options that are not used in Aviary and can be safely ignored. Variables related to mission definition are important, but Aviary defines mission profiles in a significantly different way. Currently, the user must build a new [mission definition file](../examples/simple_mission_example.ipynb) that recreates the mission.
    Aviary will ignore unconverted variables when loading the csv, so you can safely leave them.


## Aviary run_mission command

The Level 1 CLI (i.e. `run_mission` option) is designed to offer the lowest barrier to entry for new users.
Analysts who have experience with legacy tools, such as `FLOPS` and `GASP`, should find the switch from FORTRAN namelists to csv-based input decks to be straightforward. Aviary input decks allow the user the ability to set aircraft characteristics and basic mission parameters,
such as cruise Mach number and altitude, in a simple text-based format that does not require any familiarity with Python or OpenMDAO.

Aviary can then be executed by calling [`aviary run_mission`](../user_guide/aviary_commands.ipynb) with the csv input deck.
Although the order of the variables in the input deck is not important, you might find it helpful to separate the variables based on if they're used as initial guesses or in the metadata.

## First aircraft model

We have a few sample aircraft csv files in `aviary/models`. They are all `.csv` files. For example, `aircraft_for_bench_GwGm.csv` is a large single aisle aircraft mission model.

Open `aircraft_for_bench_GwGm.csv` using your favorite editor (e.g., MS Excel or VS Code). If you choose to use MS Excel, you need to take extra steps to make sure that it is in the original format. We recommend running dos2unix first (e.g. `dos2unix aircraft_for_bench_GwGm.csv`).

The file `aircraft_for_bench_GwGm.csv` is a data input file modeling large single aisle aircraft which was converted from a `GASP` input deck.
Here is a snippet of the file:

```
debug_mode,False

aircraft:controls:cockpit_control_mass_scaler,1,unitless
aircraft:controls:control_mass_increment,0,lbm
aircraft:controls:stability_augmentation_system_mass,0,lbm
......
mission:design:cruise_altitude,37500,ft
mission:design:gross_mass,175400,lbm
......
reserves,0
rotation_mass,0.99
time_to_climb,0
......
INGASP.ALR,1.11,
INGASP.BENGOB,0.05,
INGASP.CINP,0.11,
......
```

The input `.csv` file defines the aircraft and mission. In an Aviary input file you see a list of Aviary variables, their values, and units. Aviary variables are colon delimited strings (e.g. `aircraft:wing:aspect_ratio`). An Aviary variable name usually has three words. The first word is `aircraft` or `mission`, the second word is a subsystem name and the third is the variable name. Each variable has a value and units. Aviary requires units for every variable because it reduces errors and is a good engineering practice.

```{note}
If you have used the `fortran_to_aviary` tool to create a `.csv` file you must check the outputted variable values and units to ensure the file is valid and what you expect.
```

Be aware that some variables do not follow the standard naming format (e.g. `reserves`). These are used for the initial guessing of the trajectory. They are intentionally separate to prevent conflicts with some similarly named variables. They are only used once to set up the problem and then are discarded.

Finally we see a list of `GASP` variables that are not converted. Not all variables are converted to Aviary right now. They may represent some features in `FLOPS` and `GASP` that we haven't implemented in Aviary yet.

To find information about a variable (e.g. description, data type, etc.), users should read the [Variable Metadata Doc](../user_guide/variable_metadata).

There are special rules for the mapping from the input file variable names to the metadata. For example, variable `aircraft:wing:aspect_ratio` in `aircraft_for_bench_GwGm.csv` is mapped to `Aircraft.Wing.ASPECT_RATIO` in `aviary/variable_info/variable_meta_data.py`. So, the first part (e.g., `aircraft` or `mission`) is mapped to the same word but with the first letter capitalized (e.g., `Aircraft` or `Mission`). The third word is all caps (e.g., `ASPECT_RATIO`). The middle part (e.g., `wing`) is a little more complicated. In most cases, this part capitalizes the first letter (e.g., `Wing`). The following words have special mappings:

- `air_conditioning -> AirConditioning`
- `anti_icing -> AntiIcing`
- `blended_wing_body -> BWB`
- `crew_and_payload -> CrewPayload`
- `horizontal_tail -> HorizontalTail`
- `landing_gear -> LandingGear`
- `tail_boom -> TailBoom`
- `vertical_tail -> VerticalTail`

This can be summarized as to capitalize the leading letter and to capitalize the first letter after special character “`_`”.

File `aviary/variable_info/variables.py` is a variable hierarchy that is for a single mission. Each mission gets a copy of this hierarchy. Below is a snippet of this file:

In [ ]:
import aviary.api as av

# Get the path of variables.py within the aviary package
file_path = av.get_path('variable_info/variables.py')

# Read and print the first 15 lines of the file
with open(file_path, 'r') as file:
    for i in range(18):
        print(file.readline().strip('\n'))

Aviary variables are always set to default values before the input file is read in. Then Aviary will update the values based on the user-provided `.csv` input file. If you want to set different values, you can set them in the `.csv` input file.

In input `*.csv` files, the first line is usually `debug_mode` with value of `FALSE`. If you are debugging, you may set it to `TRUE` and more information will be output on the command line when you run your model. Most of the additional lines of output are from your optimizer. The `debug_mode` variable does not help a new user very much and is primarily for advanced users.

## First level 1 run

We are ready to run some models. By default, Aviary runs the optimizer for up to 50 iterations. In order to reduce the run time, we will limit the maximum number of iterations to 1. It will be faster for these examples, but you will not get optimal solutions. Issue the following command:

In [ ]:
!aviary run_mission models/test_aircraft/aircraft_for_bench_GwGm.csv --max_iter 1 --optimizer IPOPT

The argument `models/test_aircraft/aircraft_for_bench_GwGm.csv` shows where the aircraft csv file lives. This argument must be one of the following three options:

- an absolute path,
- a relative path relative to the current working directory,
- a relative path relative to the Aviary package.

Aviary searches for the given dataset in this order. If a dataset file exists in multiple directories, the first one Aviary finds will be used.

## Warning messages

During your first run you may notice some warning messages in the Aviary output. Frequently seen warnings are:

- **PromotionWarning:** Issued when there is ambiguity due to variable promotion (an [OpenMDAO warning](https://openmdao.org/newdocs/versions/latest/features/warning_control/warnings.html)).
- **RuntimeWarning:** Issued for warnings about dubious runtime features (a [Python warning](https://docs.python.org/3/library/warnings.html)).
- **UserWarning:** Issued for warnings about potential OpenMDAO, Dymos, and/or Aviary problems.
- **DerivativesWarning:** Issued when the approximated partials or coloring cannot be evaluated as expected (an [OpenMDAO warning](https://openmdao.org/newdocs/versions/latest/features/warning_control/warnings.html)).

Some of these warnings are expected and can be ignored. For example, the following warnings are expected when running the `aircraft_for_bench_GwGm.csv` model:

```
~/workspace/OpenMDAO/openmdao/core/group.py:326: PromotionWarning:<class Group>: Setting input defaults for input 'mission:takeoff:ascent_duration' which override previously set defaults for ['auto', 'prom', 'src_shape', 'val'].
~/workspace/dymos/dymos/phase/phase.py:2007: RuntimeWarning: Invalid options for non-optimal control 'throttle' in phase 'groundroll': lower, upper
  warnings.warn(f"Invalid options for non-optimal control '{name}' in phase "
~/workspace/dymos/dymos/phase/phase.py:1967: UserWarning: Phase time options have no effect because fix_initial=True or input_initial=True for phase 'traj.phases.rotation': initial_ref
  warnings.warn(f'Phase time options have no effect because fix_initial=True '
```

For now, we can ignore the warning messages and continue.
<!-- TODO: expand this section -->

## Level 1 run options
In addition to a model input file, this option has additional options that are important, as seen here:

In [ ]:
!aviary run_mission

Let us discuss these in more detail:

- `-o ––outdir`: Use specified directory to write output. The default is the current directory.

- `--optimizer`: Name of optimizer. Choices are: `SNOPT`, `IPOPT`, `SLSQP`, and `None`. The default is `SNOPT`. If optimizer is `None`, it will be set to `IPOPT` or `SNOPT` depending on the analysis scheme. The optimization objective is fuel burn for level 1 runs. The objective is
  - `mission:objectives:fuel` if `mission_method` is `GASP` 
  - `fuel_burned` if `mission_method` is `FLOPS`.

- `--phase_info`: Path to phase info file. If not provided, it is `default_phase_info/gasp.py` if Mission origin is `2DOF` and `default_phase_info/flops.py` for `simple`.

- `--n2`: Generate an n2 diagram after the analysis. “[N2](https://openmdao.org/newdocs/versions/latest/features/model_visualization/n2_basics/n2_basics.html)” diagram is an OpenMDAO helper to visualize the model (see [details](https://openmdao.org/newdocs/versions/latest/features/model_visualization/n2_details/n2_details.html) too and [tutorial](https://www.youtube.com/watch?v=42VtbX6CX3A)). It is highly recommended that this option is turned on for new users.

- `--max_iter`: Maximum number of iterations. Default is 50.

- `shooting`: Use shooting instead of collocation.

For `aircraft_for_bench_GwGm` example so far, we have used the `SNOPT` optimizer because it is the default. If we want to use the `IPOPT` optimizer, we can run:

In [ ]:
!aviary run_mission models/test_aircraft/aircraft_for_bench_GwGm.csv --optimizer IPOPT --max_iter 1

## Other available input files

Other input files reside in aviary/models. They include:

```
aviary/models/validation_data/N3CC/N3CC_generic_low_speed_polars_FLOPSinp.csv
aviary/models/large_single_aisle_1/large_single_aisle_1_GwGm.csv
aviary/models/small_single_aisle/small_single_aisle_GwGm.csv
aviary/models/test_aircraft/converter_configuration_test_data_GwGm.csv
```

For example, to run `large_single_aisle_1` model, you execute the following command:

In [ ]:
!aviary run_mission models/large_single_aisle_1/large_single_aisle_1_GwGm.csv --max_iter 1 --optimizer IPOPT

For FLOPS-derived models, let us run:

In [ ]:
!aviary run_mission models/test_aircraft/aircraft_for_bench_FwFm.csv --max_iter 1 --optimizer IPOPT

## Phase info

Aviary runs depend not only on input `*.csv` files, but also on `phase_info` dictionaries. A `phase_info` dictionary defines the mission and any analysis settings used within Aviary. It is used by Aviary to build a mission trajectory (e.g., from take-off to landing).

A [Python dictionary](https://www.w3schools.com/python/python_dictionaries.asp) is a set of key-value pairs. Most keys in `phase_info` are self-explained. Users should read Dymos' [Phase Options](https://openmdao.github.io/dymos/api/phase_api.html). For more details about phase_info keys, especially their example usages, please read [onboarding phase information](input_csv_phase_info.md).

### Default GASP phases

The `default_phase_info/two_dof.py` file is shown below:

In [ ]:
import aviary.api as av

# Get the path of variables.py within the aviary package
file_path = av.get_path('interface/default_phase_info/two_dof.py')

# Read the file contents
with open(file_path, 'r') as file:
    # Find the line that starts with 'phase_info ='
    for line in file:
        if line.startswith('phase_info ='):
            print(line.strip('\n'))
            break
    
    # Print the remaining lines
    for line in file:
        print(line.strip('\n'))


The file `default_phase_info/gasp.py` contains the following phases:

```
groundroll, rotation, ascent, accel, climb1, climb2, cruise, desc1, desc2.
```

All of the above phases belong to mission. No pre-mission phase is provided. If `pre_mission` is missing, a default `pre_mission` is provided:
```
    'pre_mission': {
        'include_takeoff': True,
        'external_subsystems': [],
    }
```

Similarly, if no post-mission phase is provided, then a default `post_mission` is provided:
```
    'post_mission': {
        'include_landing': True,
        'external_subsystems': [],
    },
```

For `GASP' missions, taxi is considered part of pre-mission and landing is considered part of post-mission.

#### Groundroll phase

Let us discuss the groundroll phase in detail as an example.

In `groundroll` phase, we are given `duration_bounds` of `((1., 100.), 's')` and a reference value `duration_ref` of `(50., 's')`. Here, `duration_bounds` is a tuple of (lower, upper) bounds for the duration of the integration variable across the phase and `duration_ref` is the unit-reference value for the duration of the integration variable across the phase (see [Dymos Variables](https://openmdao.github.io/dymos/features/phases/variables.html)). This implies a time range of `(0.2, 2)` in ground roll phase, or:
```
0.02 ≤ traj.groundroll.t_duration ≤ 2.
```
We see `velocity_lower: (0, 'kn')`, `velocity_upper: (1000, 'kn')`, and `velocity_ref: (100, 'kn')`. They result in `TAS` as a design variable with a range: `(0, 10)`, or
```
0 ≤ traj.groundroll.states:TAS ≤ 10.
```
Similarly, we get:
```
0 ≤ traj.groundroll.states:distance ≤ 3.3333
```
The mass parameter is a little bit different because we have `mass_upper: (None, 'lbm')` (and `mass_lower: (0, 'lbm')`). In this situation, OpenMDAO provides replaces the `None` with an actual upper bound that is very large. We get:
```
0 ≤ traj.groundroll.states:mass ≤ 5.7e+15.
```
Comparing to `TAS` and `distance`, `mass` is not scaled to the same range. This is okay because aircraft mass is actually a constant in this phase. A huge upper bound will not have an impact.

In ground roll phase, throttle setting is set to maximum (1.0). Aviary sets a phase parameter:
```
Dynamic.Mission.THROTTLE = 1.0
```
For the [`COLLOCATION`](https://openmdao.github.io/dymos/getting_started/collocation.html) setting, there is one [segment](https://openmdao.github.io/dymos/getting_started/intro_to_dymos/intro_segments.html) (`'num_segments': 1`) and polynomial interpolation degree is 3 (`'order': 3`). Increasing the number of segments and/or increasing the degree of polynomial will improve accuracy but will also increase the complexity of computation. For groundroll, it is unnecessary.

Since groundroll is the first phase, `connect_initial_mass` is set to `False`. `fix_initial` is set to `True`, but `fix_initial_mass` to `False` because aircraft total mass is a design parameter.

Parameter `mass_defect_ref: (10.e3, 'lbm')` (or `mass_defect_ref: (1.e4, 'lbm')`) lbm. Defect is a residual that measures how accurately the proposed state and control history obeyed the ODE governing the system dynamics. For distance, the defect reference is `100` ft. For `TAS`, `defect_ref` is not provided. The default behavior is to make make the `defect_ref` equal to the `ref` value. Please see [here](https://stackoverflow.com/questions/65243017/openmdao-dymos-defect-refs-how-should-i-set-these) for more info.

#### Other phases

We will not discuss the other phases in detail.

Phase `climb1` is for climb up to 10,000 ft and phase `climb2` is for climb to cruise phase. Phase `desc1` is for descent down to 10,000 ft and phase `desc2` is for descent from 10,000 ft down to 1,000 ft.

### Default FLOPS phases

The file `default_phase_info/flops.py` contains the following phases:

```
pre_mission, climb, cruise, descent, post_mission
```

The differences between `GASP` and `FLOPS` phases are due to how `GASP` and `FLOPS` implement trajectory analysis.

```{note}
File `default_phase_info/flops.py` has a `pre_mission` phase and a `post_mission` phase. In `pre_mission`, `takeoff` is the simplified takeoff and in `post_mission`, `landing` is the simplified landing. For `FLOPS` missions, there are [detailed takeoff and landing](../user_guide/FLOPS_based_detailed_takeoff_and_landing) available. But they are not used in `default_phase_info/flops.py`. The other phases are mission phases.
```

## Outputted files and how to read them

```{note}
Please also see [this doc page](../user_guide/outputs_and_how_to_read_them.md) for more info about the Aviary dashboard and the information is contains.
```

Each standard run generates several output files. Which output files are generated depend on the run options. In this section, we assume that we've set `max_iter = 50` in order to make sure of convergence. The following screenshots used in this article are all from a run using `aircraft_for_bench_GwGm.csv`.

First, there is always a sub-folder `reports/aviary` that contains a few HTML files.

- `driver_scaling_report.html`
- `inputs.html`
- `n2.html`
- `opt_report.html`
- `total_coloring.html`
- `traj_linkage_report.html`
- `traj_results_report.html`

### File `driver_scaling_report.html`

This file is a summary of driver scaling information. After all design variables, objectives, and constraints are declared and the problem has been set up, this report presents all the design variables and constraints in all phases as well as the objectives. The file is divided to three blocks: `Design Variables`, `Constraints`, and `Objectives`. It contains the following columns: name, size, indices, driver value and units, model value and units, ref, ref0, scaler, adder, etc. It also shows `Jacobian Info` - responses with respect to design variables (`DV`). A screen shot of design variables is shown here:

> ![Sample driver_scaling_report.html (top lines)](images/driver_scaling_report_top.PNG)

This file is needed when you are debugging. New users can skip it.

### File `inputs.html`

File `inputs.html`  is a sortable and filterable input report of input variables in different phases. It contains all the input variables (with possibly duplicate names for different phases) but only shows those in `Source is IVC` (abbreviation for [IndepVarComp](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_components/indepvarcomp.html)) on opening up. Users can choose to show other inputs by selecting and deselecting this checkbox. Users can filter their inputs by input name, source name, units, shape, tags,  values (`Val`), or design variables (`Source is DV`). Here is a screen shot (top part) when it is opened:

> ![Sample inputs.html (top lines)](images/inputs_html_top.PNG)

New users can choose to use `input_list.txt` instead (see below).

### File `n2.html`

[N2](https://openmdao.org/newdocs/versions/latest/features/model_visualization/n2_details/n2_details.html), sometimes referred to as an eXtended Design Structure Matrix (XDSM), is a powerful tool for understanding your model in OpenMDAO. It is an N-squared diagram in the shape of a matrix representing functional or physical interfaces between system elements. It can be used to systematically identify, define, tabulate, design, and analyze functional and physical interfaces.

```{note}
We *strongly* recommend that you understand N2 diagrams well, especially if you are debugging a model or adding external subsystems. Here is a [doc page](https://openmdao.github.io/PracticalMDO/Notebooks/ModelConstruction/using_n2.html) featuring an informative video on how to use N2 diagrams effectively. MDO lab has another resource for learning about N2 and [XDSM](https://mdolab-pyxdsm.readthedocs-hosted.com/).
```

 Here is a screenshot of an N2 diagram after we run `aviary run_mission models/test_aircraft/aircraft_for_bench_GwGm.csv`.

> ![Sample n2.html (on start)](images/N2_start.PNG)

At level 1, we are interested in the phases, so let us `Hide solvers`. So from `model`, zoom into `traj`, then into `phases`. We see the phases from `groundroll` to `desc2` as expected. To see how those phases are connected, we click on `Show all connections in view`. Here is what we see:

> ![Sample n2.html (on phases)](images/N2_phases.PNG)

The solid arrow connections show how outputs of one phase feed as inputs to the next. As you can see, the input of each phase is linked from the previous phase and its output is linked to the next phase. This is pretty much what we expect. The dashed arrows are links to and/or from other places not in the view. If you are curious where those links go, you must zoom out. This can be done by clicking the vertical bar on the left. Let us click it twice. Here is what we get:

> ![Sample n2.html (on phases)](images/N2_phases_links.PNG)

Note that `desc2` phase is linked back up to `static_analysis`. This means that at least one variable must be matched in the two places. There must be a nonlinear solver for this step. 

For `GASP` missions, there is a takeoff subsystem (including `taxi` phase) within `pre_mission`. For `FLOPS` missions, there is a takeoff subsystem within `pre_mission` in which a `takeoff` phase is added. It is added after `static_analysis` in `add_pre_mission_systems` method.

Similarly, there is a `landing` phase within `post_mission`.

### File `opt_report.html`

This file is OpenMDAO Optimization Report. All values are in unscaled, physical units. On the top is a summary of the optimization, followed by the objective, design variables, constraints, and optimizer settings. Here is a screenshot:

> ![Sample opt_report.html (top lines)](images/opt_report_top.PNG)

This file is important when dissecting optimal results produced by Aviary.

### Coloring files

There is a sub-folder `coloring_files`. Those are used internally by OpenMDAO for derivative computation. Users should generally skip those files.

### File `traj_linkage_report.html`

This is a dymos linkage report in a customized N2 diagram. It provides a report detailing how phases are linked together via constraint or connection. It can be used to identify errant linkages between fixed quantities.

> ![Sample traj_linkage_report.html (top of timeseries)](images/traj_linkage_report_top.PNG)

We clearly see how those mission phases are linked.

### File `traj_results_report.html`

```{note}
This is one of the most important files produced by Aviary. It will help you visualize and understand the optimal trajectory produced by Aviary.
```

This file contains timeseries and phase parameters in different tabs. For our `aircraft_for_bench_GwGm` run, they are: groundroll, rotation,  ascent, accel, climb1, climb2, cruise, desc1, and desc2 parameters. On the timeseries tab, users can select which phases to view. The following are the top of timeseries tab and ascent parameters tab:

> ![Sample traj_results_report.html (top part of timeseries)](images/traj_results_report_top.PNG)

Let's find the altitude chart. Move the cursor to the top of climb2. We see that the aircraft climbs to 37910 feet at 2814 second. Then it enters to cruise phase. At 28290 second, it starts descent from 37660 feet.

> ![Sample traj_results_report.html (altitude)](images/traj_results_report_altitude.PNG)

Let's switch to `ascent` tab. We see the following:

> ![Sample traj_results_report.html (top lines) of ascent](images/traj_results_report_top_ascent.PNG)

This file is quite important. Users should play with it and try to grasp all possible features. For example, you can hover the mouse over the solution points to see solution value; you can save the interesting images; you can zoom into a particular region for details, etc.

### Optimizer output

If `IPOPT` is the optimizer, `IPOPT.out` is generated. If `SLSQP` is the optimizer and `pyOptSparseDriver` is the driver, `SLSQP.out` is generated. Generally speaking, `IPOPT` and `SNOPT` converge Aviary optimization problems better than `SLSQP`, but `SLSQP` is bundled with Scipy by default, making it more widely available.

### SQLite database file

There is a `.db` file after run. By default, it is `aviary_history.db`. This is an SQLite database file. In level 2 and level 3, we will be able to choose a different name. Our run is recorded into this file. You generally shouldn't need to parse through this file on your own, but it is available if you're seeking additional problem information.

### Plain text inputs and outputs

If `debug_mode` is set to `True`, `input_list.txt` and `output_list.txt` are generated. We recommend this parameter be set to `True` for beginners. 

Let us look at `input_list.txt`:

> ![Sample input_list.txt (top lines)](images/input_list_top.PNG)

In this screenshot, we see a tree structure. There are three columns. The left column is a list of variable names. The middle column is the value and the right column is the [promoted](https://openmdao.org/newdocs/versions/latest/basic_user_guide/multidisciplinary_optimization/linking_vars.html) variable name. `pre_mission` is a phase, `static_analysis` is a subgroup which contains other subcomponents (e.g. `propulsion_group`). `engine` is a component under which is a list of input variables (e.g. `aircraft:engine:scaled_sls_thrust`).

An input variable can appear under different phases and within different components. Note that its values can be different because its value has been updated during the computation. On the top-left corner is the total number of inputs. That number counts the duplicates because one variable can appear in different phases. Aviary variable structure are discussed in [Understanding the Variable Hierarchy](https://github.com/OpenMDAO/Aviary/blob/main/aviary/docs/user_guide/variable_hierarchy.ipynb) and [Understanding the Variable Metadata](https://github.com/OpenMDAO/Aviary/blob/main/aviary/docs/user_guide/variable_metadata.ipynb).

If you zoom the N2 diagram into `propulsion_group` and `gasp_based_geom`, you see the exact same tree structure:

> ![Sample N2 Diagram (top blocks)](images/N2_top.PNG)

File `output_list.txt` follows the same pattern. But there is another tree for implicit outputs. That helps when debugging is needed.

### Additional messages

When Aviary is run, some messages are printed on the command line and they are important. For example, the following constraint report tells us whether our desired constraints are met:

```
--- Constraint Report [traj] ---
    --- groundroll ---
        None
    --- rotation ---
        [final]   0.0000e+00 == normal_force [lbf]
    --- ascent ---
        [final]   5.0000e+02 == altitude [ft]
        [path]    0.0000e+00 <= load_factor <= 1.1000e+00  [unitless]
        [path]    0.0000e+00 <= fuselage_pitch <= 1.5000e+01  [deg]
    --- accel ---
        [final]   2.5000e+02 == EAS [kn]
    --- climb1 ---
        [final]   1.0000e+04 == altitude [ft]
    --- climb2 ---
        [final]   3.7500e+04 == altitude [ft]
        [final]   1.0000e-01 <= altitude_rate  [ft/min]
        [final]   8.0000e-01 == mach [unitless]
    --- cruise ---
        None
    --- desc1 ---
        [final]   1.0000e+04 == altitude [ft]
    --- desc2 ---
        [final]   1.0000e+03 == altitude [ft]
```

## Beyond level 1

Aviary Level 1 is quite capable yet limited in how much is exposed to the user. If you need additional flexibility or want to add your own subsystems, you will need to move to Levels 2 or 3. Let us briefly discuss some of the possible reasons for you to move to Levels 2 or 3.

If you examine the Aviary metadata, you find that Aviary has two choices of objectives: `Mission.Objectives.FUEL` and `Mission.Objectives.RANGE`. But in level 1, you are limited to minimizing fuel. Users will be able to make other choices in level 2 and level 3.

Level 1 uses the default `phase_info` dictionaries, though there is an option to use a customized `phase_info`.
For more details, please read [drawing and running simple missions](../user_guide/drawing_and_running_simple_missions).

There may be more or fewer key-value pairs in each of the phases. For example, if you have an external subsystem, say, a battery with `BatteryBuilder()`, you can add a key `external_subsystems` with value `BatteryBuilder()`. We will go into details later.

No matter what mission method users choose, they can create their own phase sequences and run Aviary using either level 2 or 3 interface.

In the `aviary/interface/default_phase_info` subdirectory, you see another `phase_info` file `solved.py`. It defines the `phase_info` object for the "solved" mission method which is not explained in level 1. The “solved” mission method is a more simplified and robust method than the other methods. In just a few words, it's a way of implementing a trajectory where the aircraft only flies in straight lines in the altitude-range space and usually at constant velocity. The idea is to simplify the design space by reducing the flexibility the optimizer has to choose a trajectory profile. It still uses a collocation method to solve the mission dynamics, but the complexity of the design problem is lessened.

We are ready to move on to [Level 2](onboarding_level2.ipynb).